In [1]:
from polls import *
from UKGE import *
import numpy as np
import datetime

In [2]:
df = pd.read_csv("outputs/resultsclusteredconstituencies.csv")

In [3]:
#Predicting 2019
poll_avg19 = get_weighted_poll_avg(url_19, col_dict=col_dict19)
poll_avg17 = get_weighted_poll_avg(url_17, col_dict=col_dict17)
poll_avg15 = get_weighted_poll_avg(url_15, col_dict=col_dict15)
poll_avg10 = get_weighted_poll_avg(url_10, col_dict=col_dict10)
poll_avg05 = get_weighted_poll_avg(url_05, col_dict=col_dict05)
poll_avg19 = standardise_df(poll_avg19, col_dict19)
poll_avg17 = standardise_df(poll_avg17, col_dict17)
poll_avg15 = standardise_df(poll_avg15, col_dict15)
poll_avg10 = standardise_df(poll_avg10, col_dict10)
poll_avg05 = standardise_df(poll_avg05, col_dict05)



In [4]:

gb19 = get_GB_res(df)
pc_res_19 = convert_to_pc(gb19)
gb17 = get_GB_res(df,yr=2017)
pc_res_17 = convert_to_pc(gb17)
gb15 = get_GB_res(df,yr=2015)
pc_res_15 = convert_to_pc(gb15)
gb10 = get_GB_res(df,yr=2010)
pc_res_10 = convert_to_pc(gb10)
gb05 = get_GB_res(df,yr=2005)
pc_res_05 = convert_to_pc(gb05)

In [5]:
poll_avgs = [poll_avg05,poll_avg10,poll_avg15,poll_avg17,poll_avg19]
pc_results = [pc_res_05,pc_res_10,pc_res_15,pc_res_17,pc_res_19]

In [6]:
bigerrs = []
for p,r in zip(poll_avgs,pc_results):
    conerr = float(p["con"] - r["con"])
    laberr = float(p["lab"] - r["lab"])

    bigerrs.append(conerr)
    bigerrs.append(laberr)






/tmp/ipykernel_67467/1519193469.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  conerr = float(p["con"] - r["con"])
/tmp/ipykernel_67467/1519193469.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  laberr = float(p["lab"] - r["lab"])


In [7]:
big_std = np.std(bigerrs) * 1.1

In [8]:
pc_res_05

,con,lab,lib,ref,nat,oth
0,0.332274,0.36199,0.226456,0.0,0.022213,0.057067


In [9]:
pd.DataFrame(poll_avg05).transpose().columns

Index(['Con', 'Lab', 'Lib Dem', 'Others', 'con', 'lab', 'lib', 'oth'], dtype='object')

In [10]:
serrs = []
for p,r in zip(poll_avgs,pc_results):
    liberr = float(p["lib"] - r["lib"])
    serrs.append(liberr)
    if "ref" in pd.DataFrame(p).transpose().columns:
        referr = float(p["ref"] - r["ref"])
        serrs.append(referr)
        
    if "nat" in pd.DataFrame(p).transpose().columns:
        naterr = float(p["nat"] - r["nat"])
        serrs.append(naterr)




/tmp/ipykernel_67467/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_67467/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_67467/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_67467/2919726663.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  referr = float(p["ref"] - r["ref"])
/tmp/ipykernel_67467/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a

In [11]:
small_std = np.std(serrs) *1.1

In [12]:
print(big_std)
print(small_std)

0.023242414561510088
0.015989670244894515


In [13]:
constituencies = list(df["New constituency name"])

In [14]:
next_poll_avg = get_weighted_poll_avg(next_url, col_dict=next_col_dict)
nat_polls = standardise_df(next_poll_avg, next_col_dict)
chg_df = nat_polls - pc_res_19
chg_df = chg_df.dropna(axis=1).copy()

In [15]:
chg_df

,con,lab,lib,nat,oth,ref
0,-0.225489,0.109106,-0.03195,-0.015484,0.016707,0.086444


In [16]:
natchgs = []
sims_num = 1000
future_uncertainty = 3
for p in std_parties:
    if nat_polls[p] > 0.2:
        natchgs.append(np.random.normal(chg_df[p],big_std*future_uncertainty,sims_num))
    else:
        natchgs.append(np.random.normal(chg_df[p],small_std*future_uncertainty,sims_num))
clusters = list(set(list(df["Cluster"])))
for p in std_parties:
    for c in clusters:
        if nat_polls[p] > 0.2:
            natchgs.append(np.random.normal(0,big_std,sims_num))
        else:
            natchgs.append(np.random.normal(0,small_std,sims_num))

In [17]:
chg_dict = {}


In [18]:
i=0
for p in std_parties:
    chg_dict[p+"n"] = natchgs[i]
    i+=1
for c in clusters:
    for p in std_parties:
        chg_dict[p+str(c)] = natchgs[i]
        i+=1

In [19]:
rand_df = pd.DataFrame(chg_dict)

In [20]:
# Create a dictionary to store the results for each constituency and party
results = {}

# Iterate over constituencies
for con in constituencies:
    # Filter the dataframe for the current constituency
    con_df = df[df["New constituency name"] == con].copy()
    
    # Get the cluster, imputation uncertainty, and future uncertainty for the current constituency
    cluster = con_df["Cluster"].iloc[0]
    imputation_uncertainty = con_df["Low_Confidence_Imputation"].iloc[0]
    
    # Iterate over standard parties
    for p in std_parties:
        # Get the previous results for the current party and constituency
        prev_res = con_df["2019_Results"+p+"_pc"]
        
        # Get the necessary data from the random dataframe
        nat_chg = rand_df[p+"n"]
        clr_chg = rand_df[p+str(cluster)]
        
        # Calculate the noise for the current constituency and party
        con_noise = [np.random.normal(0, (1+imputation_uncertainty)*small_std) for n in range(sims_num)]
        
        # Calculate the rough prediction for the current constituency and party
        prev_res_array = [float(prev_res.iloc[0]) for n in range(sims_num)]
        rough_pred = prev_res_array + nat_chg + clr_chg + con_noise
        rough_pred[rough_pred < 0] = 0
        
        # Store the results in the dictionary
        results[con+p+"Noise"] = con_noise
        results[con+p+"Rough"] = rough_pred
    
# Update the random dataframe with the results
rand_df = pd.concat([rand_df,pd.DataFrame(results)],axis=1)

In [21]:
def get_winner(party_values, winner_val,parties = std_parties):
    for p,v in zip(parties,list(party_values)[0]):
        if v == winner_val:
            return p
def get_margin(party_values):
    sorted_vals = sorted(list(party_values)[0])
    margin = sorted_vals[-1] - sorted_vals[-2]
    return margin

final_dfs = [rand_df]
for con in constituencies:
    con_cols = []
    for p in std_parties:
        con_cols.append(con+p+"Rough")
    final_df = rand_df[con_cols].copy()
    final_pred_cols = []
    for p in std_parties:
        final_df[con+p+"Final"] = final_df[con+p+"Rough"] / final_df[con_cols].sum(axis=1)
        final_pred_cols.append(con+p+"Final")
    final_df[con +"past_post_val"] = final_df[final_pred_cols].max(axis=1)
    final_df[con+"Margin"] = final_df[final_pred_cols].apply(lambda x:get_margin([x[final_pred_cols]]),axis=1)
    final_pred_cols.append(con +"past_post_val")
    final_df[con+"Winner"] = final_df[final_pred_cols].apply(lambda x:get_winner([x[final_pred_cols]],x[con+"past_post_val"]),axis=1)

                
    # final_df[con+"Winner"] = winners
    final_dfs.append(final_df)
rand_df = pd.concat(final_dfs, axis=1)
    

In [22]:
seats_cols = []
for p in std_parties:
    rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
    seats_cols.append(p+"Seats")

/tmp/ipykernel_67467/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/tmp/ipykernel_67467/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/tmp/ipykernel_67467/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining 

In [23]:
rand_df[seats_cols]

,conSeats,labSeats,libSeats,refSeats,natSeats,othSeats
0,199,367,23,0,42,1
1,244,313,19,0,56,0
2,190,342,38,1,59,2
3,222,324,29,0,56,1
4,254,312,10,0,55,1
...,...,...,...,...,...,...
995,262,313,16,0,41,0
996,36,496,53,0,46,1
997,226,347,18,0,41,0
998,238,315,17,3,57,2


In [24]:
max_seats = rand_df[seats_cols].max(1)
results = []
for n in range(sims_num):
    for col in seats_cols:
        if max_seats[n] ==rand_df[col].loc[n]:
            for p in std_parties:
                if p + "Seats" == col:
                    winning_party = p
    
    if max_seats[n] <326:
        result = "Hung Parliament: Largest Party " + winning_party
    else:
        result = winning_party
    results.append(result)
rand_df["Result"] = results


/tmp/ipykernel_67467/3465883602.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df["Result"] = results


In [25]:
rand_df["Result"].value_counts()

Result
lab                                   764
Hung Parliament: Largest Party lab    203
Hung Parliament: Largest Party con     29
con                                     4
Name: count, dtype: int64

In [26]:
print(nat_polls["lab"] - nat_polls["con"])

0.21383333333333324


In [27]:
nat_polls

Con         0.234500
Lab         0.448333
Lib Dems    0.089833
SNP         0.030500
Green       0.061167
Reform      0.107667
Others      0.028500
con         0.234500
lab         0.448333
lib         0.089833
ref         0.107667
nat         0.030500
oth         0.028500
dtype: float64

In [28]:
winner_cols = [con+"Winner" for con in constituencies]

In [29]:
winner_wins = []
for col in winner_cols:
    winner_wins.append(max(list(rand_df[col].value_counts())))

In [30]:
close_cols = []
for col,wins in zip(winner_cols,winner_wins):
    if wins < 525:
        close_cols.append(col)

In [31]:
rand_df[close_cols]

,AylesburyWinner,Bicester and WoodstockWinner,"Brecon, Radnor and Cwm TaweWinner",CongletonWinner,Didcot and WantageWinner,EastleighWinner,Ely and East CambridgeshireWinner,Gillingham and RainhamWinner,GraveshamWinner,Harpenden and BerkhamstedWinner,HitchinWinner,MansfieldWinner,Mid DerbyshireWinner,Rochester and StroodWinner,St Neots and Mid CambridgeshireWinner,StaffordWinner,Stoke-on-Trent SouthWinner,Sutton and CheamWinner,WokingWinner,MidlothianWinner
0,lab,lab,lab,con,lib,con,lab,con,con,con,lab,con,con,con,con,lab,con,lib,lab,lab
1,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,nat
2,lab,con,lib,con,lib,lib,con,con,con,lib,con,con,con,con,con,lab,con,lib,lib,nat
3,con,con,con,con,lib,con,lib,con,con,con,con,con,con,con,con,con,con,lib,lib,nat
4,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,nat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,lab
996,lab,lib,lab,lab,lib,lib,lib,lab,lab,lab,lib,lab,lab,lab,lab,lab,lab,lib,lib,nat
997,con,lab,con,con,con,con,con,con,con,con,con,con,con,con,con,con,lab,con,con,lab
998,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,nat


In [32]:
all_final_cols = [con+p+"Final"for p in std_parties for con in constituencies]
all_margin_cols = [con+"Margin"for p in std_parties for con in constituencies]
winner_cols
results_cols = ["Result"]


In [33]:
seats_cols

['conSeats', 'labSeats', 'libSeats', 'refSeats', 'natSeats', 'othSeats']

In [34]:
all_result_cols = []
for l in [all_final_cols,all_margin_cols,winner_cols,seats_cols,results_cols]:
    all_result_cols.extend(l)

In [35]:
results_df = rand_df[all_result_cols].transpose().copy()

In [36]:
results_df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
Aberafan MaestegconFinal,0.045443,0.027779,0.03477,0.0,0.10544,0.066375,0.049491,0.0,0.085733,0.036173,...,0.0,0.0,0.019812,0.104806,0.012357,0.117664,0.0,0.10625,0.069185,0.0
AldershotconFinal,0.438884,0.524461,0.375423,0.423394,0.465716,0.468523,0.322528,0.30663,0.413358,0.373289,...,0.377802,0.370099,0.402404,0.447242,0.274693,0.448499,0.358082,0.505758,0.443586,0.211636
Aldridge-BrownhillsconFinal,0.438952,0.56579,0.474871,0.538541,0.529865,0.462089,0.374559,0.350932,0.476058,0.451801,...,0.417142,0.453221,0.430611,0.512835,0.353018,0.557215,0.417696,0.528866,0.582482,0.303341
Altrincham and Sale WestconFinal,0.270063,0.376026,0.243621,0.293492,0.343611,0.329895,0.219659,0.202519,0.296503,0.243495,...,0.27762,0.213105,0.253023,0.352569,0.176106,0.378687,0.217759,0.373343,0.312203,0.08706
Alyn and DeesideconFinal,0.191356,0.186213,0.179537,0.178766,0.263837,0.231812,0.159345,0.107826,0.213368,0.20641,...,0.132136,0.177529,0.168599,0.249075,0.144098,0.274263,0.099315,0.328101,0.268996,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
libSeats,23,19,38,29,10,14,27,30,19,22,...,24,60,45,21,43,16,53,18,17,44
refSeats,0,0,1,0,0,0,0,0,0,0,...,0,0,3,2,0,0,0,0,3,0
natSeats,42,56,59,56,55,47,52,34,58,43,...,21,46,53,59,52,41,46,41,57,59
othSeats,1,0,2,1,1,1,2,2,0,2,...,1,2,4,2,2,0,1,0,2,3


In [37]:
today = datetime.date.today()

rand_df[all_result_cols].transpose().to_csv("outputs/results/Results_"+str(today)+".csv")

In [38]:
all_margin_cols

['Aberafan MaestegMargin',
 'AldershotMargin',
 'Aldridge-BrownhillsMargin',
 'Altrincham and Sale WestMargin',
 'Alyn and DeesideMargin',
 'Amber ValleyMargin',
 'Arundel and South DownsMargin',
 'AshfieldMargin',
 'AshfordMargin',
 'Ashton-under-LyneMargin',
 'AylesburyMargin',
 'BanburyMargin',
 'Bangor AberconwyMargin',
 'BarkingMargin',
 'Barnsley NorthMargin',
 'Barnsley SouthMargin',
 'Barrow and FurnessMargin',
 'Basildon and BillericayMargin',
 'BasingstokeMargin',
 'BassetlawMargin',
 'BathMargin',
 'BatterseaMargin',
 'BeaconsfieldMargin',
 'Beckenham and PengeMargin',
 'BedfordMargin',
 'Bermondsey and Old SouthwarkMargin',
 'Bethnal Green and StepneyMargin',
 'Beverley and HoldernessMargin',
 'Bexhill and BattleMargin',
 'Bexleyheath and CrayfordMargin',
 'Bicester and WoodstockMargin',
 'BirkenheadMargin',
 'Birmingham EdgbastonMargin',
 'Birmingham ErdingtonMargin',
 'Birmingham Hall Green and MoseleyMargin',
 'Birmingham Hodge Hill and Solihull NorthMargin',
 'Birmingha